In [1]:
%%javascript
$('#appmode-leave').hide();
$('#visit-repo-link').hide();

<IPython.core.display.Javascript object>

Copyright **Peter Kraus and Paolo Raiteri**, January 2021

## Determination of the Rate Law \#5
The gas phase decomposition of ethane at 856 K
\begin{equation*}
C_2H_6 \to C_2H_4 + H_2
\end{equation*}
was investigated by following the change in total pressure with time at constant volume.

Perform a series of virtual experiments to determine the order of the reaction and the rate constant at this temperature.

### Instructions:

- Use the slide bar below to select  the times at which you measure the pressure in the vessel, 
- Click `Perform measurement` to run the virtual experiment and obtain the result of the experiment,
- Click `Download CSV` to export the complete data set for all the experiments as a CSV file.

In [2]:
import ipywidgets as ipw
import json
import random
import time
import pandas as pd
import os
import webbrowser
import math
from IPython.display import display, Markdown

In [3]:
# set kinetic parameters
with open("rate_parameters.json") as infile:
    jsdata = json.load(infile)

params = jsdata["kin5"]


In [4]:
# delete existing result file and setup rng
if os.path.exists(os.path.join(os.getcwd(), "results.csv")):
    os.remove(os.path.join(os.getcwd(), "results.csv"))

#random.seed(params["error"].get("seed", 0))
t = int( time.time() * 1000.0 )
random.seed( ((t & 0xff000000) >> 24) +
             ((t & 0x00ff0000) >>  8) +
             ((t & 0x0000ff00) <<  8) +
             ((t & 0x000000ff) << 24)   )

initialPressure = [1]
def init():
    global initialPressure
    initialPressure[0] = 10 * random.random()


In [6]:
out_R = ipw.Label(value="Click Calculate.")
out_P = ipw.Output()
out_L = ipw.Output()

result = "Pressure (bar)"
value = "Time (hour)"

with out_L:
    display(Markdown("[Download CSV](./results.csv)"))
    
def calc(btn):
    R = initialPressure[0] * (2 - math.exp(-params["k"] * time.value))
    
    # Random error
    dR = R * params["error"].get("random", 0) * (0.5 - random.random()) * 2 
    # Systematic error
    dR = dR + R * params["error"].get("systematic", 0)
    out_R.value = f"{R + dR:.6e}"
    out_P.clear_output()
    try:
        res = pd.read_csv("results.csv") 
    except FileNotFoundError:
        res = pd.DataFrame(columns=[value,
                                    result])
    res = res.append({value: time.value, 
                      result: R + dR}, 
                     ignore_index=True)
    res.to_csv("results.csv", index=False)
    with out_P:
        display(res.tail(50))

def reset(btn):
    if os.path.exists(os.path.join(os.getcwd(), "results.csv")):
        os.remove(os.path.join(os.getcwd(), "results.csv"))
    res = pd.DataFrame(columns=[value,
                                result])
    res.to_csv("results.csv", index=False)
    with out_P:
        out_P.clear_output()
        display(res.tail(50))
    init()

time = ipw.FloatLogSlider(value=params["time"], min=0, max=4)

btn_reset = ipw.Button(description="Restart Laboratory", layout=ipw.Layout(width="150px"))
btn_reset.on_click(reset)


btn_calc = ipw.Button(description="Perform measurement", layout=ipw.Layout(width="150px"))
btn_calc.on_click(calc)

rows = []
rows.append(ipw.HBox([ipw.Label(value=f"Elapsed time:", 
                                layout=ipw.Layout(width="250px")),
                      time,
                      ipw.Label(value="hour")]))

rows.append(ipw.HBox([ipw.Label(value="Pressure Measured:", 
                                layout=ipw.Layout(width="250px")), 
                      out_R,
                      ipw.Label(value="bar")]))

rows.append(ipw.HBox([btn_reset, btn_calc, out_L]))
rows.append(ipw.HBox([out_P]))
reset(btn_reset)
ipw.VBox(rows)